# Minio with Apache Arrow

## 1 Compare performance reading Arrow from Minio and creating Spark dataframe

In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
from pyarrow import *
import pandas as pd
import numpy as np
import os
import psutil
import pyspark
import sys

In [2]:
%env AWS_S3_ENDPOINT_URL=http://127.0.0.1:9000
%env AWS_ACCESS_KEY_ID=1B80PP32Z5VV6NJA6ABE
%env AWS_SECRET_ACCESS_KEY=fo0mwSE+MAWmNdhNJH0cY8qxXcj+sxbeKrZCk6Vl

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.11.534,org.apache.hadoop:hadoop-aws:2.7.3,org.apache.httpcomponents:httpclient:4.3.3 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.minio:spark-select_2.11:2.1,org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell'
os.environ["PYSPARK_PYTHON"] = "/Users/ravishankarnair/anaconda3/bin/python"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.9-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master("spark://ravion.local:7077")  \
        .appName("Minio-Arrow-Spark") \
        .getOrCreate()
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://10.0.0.251:9000");

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "1B80PP32Z5VV6NJA6ABE");

sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key","fo0mwSE+MAWmNdhNJH0cY8qxXcj+sxbeKrZCk6Vl");

sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access","true");

sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false");

sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem");


       
spark

#.config("spark.driver.extraClassPath", "/Users/ravishankarnair/lib/s3/hadoop-aws-2.7.3.jar:/Users/ravishankarnair/lib/s3/aws-java-sdk-s3-1.11.856.jar:/Users/ravishankarnair/lib/s3/aws-java-sdk-1.11.856.jar:/Users/ravishankarnair/lib/s3/aws-java-sdk-core-1.11.856.jar") \



env: AWS_S3_ENDPOINT_URL=http://127.0.0.1:9000
env: AWS_ACCESS_KEY_ID=1B80PP32Z5VV6NJA6ABE
env: AWS_SECRET_ACCESS_KEY=fo0mwSE+MAWmNdhNJH0cY8qxXcj+sxbeKrZCk6Vl


In [5]:
from pyspark.sql.types import *

sdf = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("s3://arrowbucket/ratings.csv")
spark.conf.set("spark.sql.execution.arrow.enabled", "false") 
%time pdf = sdf.toPandas()

CPU times: user 444 ms, sys: 33.4 ms, total: 477 ms
Wall time: 1.15 s


In [8]:
sdf = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("s3://arrowbucket/ratings.csv")
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
%time pdf = sdf.toPandas()


Py4JJavaError: An error occurred while calling o168.load.
: org.apache.hadoop.fs.s3.S3Exception: org.jets3t.service.S3ServiceException: S3 Error Message. -- ResponseCode: 403, ResponseStatus: Forbidden, XML Error Message: <?xml version="1.0" encoding="UTF-8"?><Error><Code>InvalidAccessKeyId</Code><Message>The AWS Access Key Id you provided does not exist in our records.</Message><AWSAccessKeyId>1B80PP32Z5VV6NJA6ABE</AWSAccessKeyId><RequestId>7B6902AB1A6131BE</RequestId><HostId>597fIxG5zNmwsBrQOnRluWPI4pi5KglRfzIv89QeVPuRToD2XGTdGlAdT3U58o96on2iLsbBOZo=</HostId></Error>
	at org.apache.hadoop.fs.s3.Jets3tFileSystemStore.get(Jets3tFileSystemStore.java:175)
	at org.apache.hadoop.fs.s3.Jets3tFileSystemStore.retrieveINode(Jets3tFileSystemStore.java:221)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at ja

### 1.1 Load and prepare data 

In [12]:
## Read movielens dataset from grouplens
df = pd.read_csv("/Users/ravishankarnair/data/ratings.csv")

### 1.2 Write to disk 

In [13]:
# Write to parquet
df.to_parquet("ratings.parquet")

context = pa.default_serialization_context()

# Write to Arrow
# Convert from pandas to Arrow
table = pa.Table.from_pandas(df)
# Write out to file

writer = pa.RecordBatchFileWriter('ratings.arrow', table.schema)
writer.write(table)
writer.close()
#with pa.OSFile('penguin-dataset.arrow', 'wb') as sink:
    #with pa.RecordBatchFileWriter(sink, table.schema,write_legacy_format=True) as writer:
        #writer.write_table(table)

/Users/ravishankarnair/anaconda3/envs/py36/lib/python3.6/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


### 1.3 Reading time - calculate average of numeric column

#### 1.3.1 Read csv and calculate mean

In [14]:
%%timeit
pd.read_csv("/Users/ravishankarnair/data/ratings.csv")["rating"].mean()

41 ms ± 902 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### 1.3.2 Read parquet and calculate mean

In [15]:
%%timeit
pd.read_parquet("ratings.parquet", columns=["rating"]).mean()

/Users/ravishankarnair/anaconda3/envs/py36/lib/python3.6/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/Users/ravishankarnair/anaconda3/envs/py36/lib/python3.6/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)


6.81 ms ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 1.3.3 Read Arrow using file API

In [16]:
%%timeit
with pa.OSFile('ratings.arrow', 'rb') as source:
    table = pa.ipc.open_file(source).read_all().column("rating")
result = table.to_pandas().mean()

2.27 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np
import os
import psutil
import sys

with pa.OSFile('/Users/ravishankarnair/scalaexp/sparkarrow/data/arrow877093401.arrow', 'rb') as source:
    table = pa.ipc.open_file(source).read_all()
result = table.to_pandas()

result.head()

#### 1.3.4 Read Arrow with memory-mapped API

In [17]:
%%timeit
source = pa.memory_map('ratings.arrow', 'r')
table = pa.ipc.RecordBatchFileReader(source).read_all().column("rating")
result = table.to_pandas().mean()

549 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 1.4 Memory consumption - read column

In [18]:
# Measure initial memory consumption
memory_init = psutil.Process(os.getpid()).memory_info().rss >> 20

#### 1.4.1 Read csv

In [19]:
col_csv = pd.read_csv("/Users/ravishankarnair/data/ratings.csv")["rating"]
memory_post_csv = psutil.Process(os.getpid()).memory_info().rss >> 20

#### 1.4.2 Read parquet

In [20]:
col_parquet = pd.read_parquet("ratings.parquet", columns=["rating"])
memory_post_parquet = psutil.Process(os.getpid()).memory_info().rss >> 20

#### 1.4.3 Read Arrow using file API

In [21]:
with pa.OSFile('ratings.arrow', 'rb') as source:
    col_arrow_file = pa.ipc.open_file(source).read_all().column("rating").to_pandas()
memory_post_arrowos = psutil.Process(os.getpid()).memory_info().rss >> 20

#### 1.4.4 Read Arrow with memory-mapped API 

In [22]:
source = pa.memory_map('ratings.arrow', 'r')
table_mmap = pa.ipc.RecordBatchFileReader(source).read_all().column("rating")
col_arrow_mapped = table_mmap.to_pandas()
memory_post_arrowmmap = psutil.Process(os.getpid()).memory_info().rss >> 20

#### 1.4.6 Display memory consupmtion

In [23]:
# Print memory consumption
print(f"csv: {memory_post_csv - memory_init}\n"
      f"Parquet: {memory_post_parquet - memory_post_csv}\n"
      f"Arrow file API: {memory_post_arrowos - memory_post_parquet}\n"
      f"Arrow memory-mapped API(zero-copy): {memory_post_arrowmmap - memory_post_arrowos}\n")
      

csv: 1
Parquet: -1
Arrow file API: 4
Arrow memory-mapped API(zero-copy): 0



In [7]:
import s3fs
from s3fs import S3FileSystem
s3 = s3fs.S3FileSystem()



In [8]:
f = s3.open_input_stream('arrowbucket/ratings.arrow')

AttributeError: 'S3FileSystem' object has no attribute 'open_input_stream'